In [13]:
import os
from pathlib import Path
import sys
import os
import _pickle as pickle
import json
import copy
import language_check
import re

In [14]:
preds = []

In [15]:
with open('./raw/predictionsRet.json') as json_file:
    sessions = json.load(json_file)

In [16]:
sessions[0][0]

{'wizard_of_wikipedia': 'Elvis Presley',
 'WizardTransformerRanker': {'text': 'Oh baby.... Elvis Presley is truly The King of Rock and Roll!',
  'text_candidates': ['Oh baby.... Elvis Presley is truly The King of Rock and Roll!',
   'I just love listening to the KIng of Rock and Roll - Elvis Presley!',
   'In 1956, Elvis released his first single called "Heartbreak Hotel."  It instantly became a number one hit.  His popularity led him to tv appearances and even movies.  ',
   'Yeah, I love the King of Rock and Roll. Is he your favorite musical artist?',
   'I was 10 when he died, so that may be why.  I visited Graceland with family, and learned about the Memphis Mafia, though!']}}

In [17]:
ind = 0
for session in sessions:
    conv = {}
    firstDiag = session[0]['wizard_of_wikipedia']
    if(len(firstDiag.split("\n")) < 2):
        continue
    conv['index'] = ind
    ind += 1
    
    conv['conversation'] = session
    
    preds.append(conv)

In [18]:
with open('predictionsRetEdit.json', 'w') as outfile:
    json.dump(preds, outfile)

In [13]:
preds = []
with open('predictionsGen.json') as json_file:
    sessions = json.load(json_file)

In [14]:
ind = 0
for session in sessions:
    conv = {}
    
    conv['index'] = ind
    ind += 1
    
    conv['conversation'] = session
    
    preds.append(conv)

In [15]:
with open('predictionsGenEdit.json', 'w') as outfile:
    json.dump(preds, outfile)

In [40]:

for root, dirs, files in os.walk("./selected/ret"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                index = 0
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        d['index'] = index
                        index += 1

                        del d['origIndex']
                    
                    filePath = './final/ret/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)

In [41]:

for root, dirs, files in os.walk("./selected/gen"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                index = 0
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        d['index'] = index
                        index += 1

                        del d['origIndex']
                    
                    filePath = './final/gen/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)

In [41]:
filePathGen = './final/gen/' + 'threeContext.json'

with open(filePathGen) as json_file:
    gen = json.load(json_file)
    
    
filePath = './final/ret/' + 'threeContext.json'

with open(filePath) as json_file:
    ret = json.load(json_file)    

In [42]:
for i in range(len(ret)):
    gen[i]['actualResponse'] = ret[i]['actualResponse']
    gen[i]['suggestions'] = [gen[i]['suggestions']]

In [43]:
with open(filePathGen, 'w') as outfile:
    json.dump(gen, outfile)

In [64]:
filePath = './clean/cleanPredictionsRet3.json'

with open(filePath) as json_file:
    ret = json.load(json_file)    

In [47]:
tool = language_check.LanguageTool('en-US')
fix_spaces = re.compile(r'\s*([?!.,\']+(?:\s+[?!.,\']+)*)\s*')

In [48]:
for root, dirs, files in os.walk("./final/gen"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        text = d['actualResponse']
                        text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()
                        matches = tool.check(text)
                        corr = language_check.correct(text, matches)
                        d['actualResponse'] = corr
                        
                        newSuggs = []
                        
                        text = d['suggestions'][0]
                        
                        text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()
                        matches = tool.check(text)
                        corr = language_check.correct(text, matches)
                        newSuggs.append(corr)
                            
                    
                        d['suggestions'] = newSuggs
                    
                    filePath = './final/gen/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)

In [49]:
for root, dirs, files in os.walk("./final/ret"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        text = d['actualResponse']
                        text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()
                        matches = tool.check(text)
                        corr = language_check.correct(text, matches)
                        d['actualResponse'] = corr
                        
                        newSuggs = []
                        
                        for text in d['suggestions']:
                        
                            text = fix_spaces.sub(lambda x: "{} ".format(x.group(1).replace(" ", "")), text).strip()
                            matches = tool.check(text)
                            corr = language_check.correct(text, matches)
                            newSuggs.append(corr)
                            
                    
                        d['suggestions'] = newSuggs
                    
                    filePath = './final/ret/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)